# Data Visualization with Vega

A picture is worth a thousand words. In machine learning, we usually handle high-dimensional data, which is impossible to draw on display directly. But a variety of statistical plots are tremendously valuable for us to grasp the characteristics of many data points. Smile provides data visualization tools such as plots and maps for researchers to understand information more easily and quickly.

Vega is a visualization grammar, a declarative language for creating, saving, and sharing interactive visualization designs. With Vega, you can describe the visual appearance and interactive behavior of a visualization in a JSON format, and generate web-based views using Canvas or SVG.

Vega-Lite is a high-level grammar of interactive graphics. It provides a concise JSON syntax for rapidly generating visualizations to support analysis. Vega-Lite specifications can be compiled to Vega specifications.

Smile provides some interactive statistical plots with latest Vega-Lite.

First, let's import Smile and also add an `implict` function to display Vega-Lite specification (a JSON object) in almond.

In [ ]:
import $ivy.`com.github.haifengl::smile-scala:2.1.0`
import $ivy.`org.slf4j:slf4j-simple:1.7.26`  

import java.lang.Math._
import smile.json._
import smile.plot.vega._
import smile.plot.show
import smile._

implicit def display(spec: JsObject): Unit = {
  publish.html(iframe(spec))
}

Now let's plot a heart. Math is beautiful, isn't it?

In [ ]:
val heart = -314 to 314 map { i =>
    val t = i / 100.0
    val x = 16 * pow(sin(t), 3)
    val y = 13 * cos(t) - 5 * cos(2*t) - 2 * cos(3*t) - cos(4*t)
    Array(x, y)
}

show(plot(heart.toArray))

Note that the function `plot` returns a `JsObject` that encapsulates the plot specification. The function `show` does the renderring job (with the help of implict argument `display` that we defined earlier).

## Scatter Plot

A scatter plot displays data as a collection of points. The points can be color-coded, which is very useful for classification tasks. The user can use `plot` functions to draw scatter plot easily.
```
  def plot(data: Array[Array[Double]],
           fields: (String, String) = ("x", "y"),
           color: Option[(String, Either[Array[Int], Array[String]])] = None,
           shape: Option[(String, Either[Array[Int], Array[String]])] = None,
           sizeOrText: Option[(String, Either[Array[Double], Array[String]])] = None,
           properties: JsObject = JsObject()): JsObject
```
The optional arguments `color`, `shape`, and `sizeOrText` take a tuple of 2 elements. The first element is the name of field, used in the legend of plot. The second element is either an array of integers (e.g. the class labels) or an array of strings (e.g. "red", "blue", etc.)

In [ ]:
val iris = read.arff("data/weka/iris.arff")
val attributes = iris.names
val x = iris.select(0, 1).toArray
val y = iris("class").toIntArray
show(plot(x, fields = (attributes(0), attributes(1)), color = Some("class" -> Left(y))))

In this example, we plot the first two columns of Iris data. We use the class label for legend and color coding. However, the Iris data has four attributes. A general practice is plot all the attribute pairs.

In [ ]:
show(spm(iris, clazz=Some("class")))

## Box Plot

The box plot is a standardized way of displaying the distribution of data based on the five number summary: minimum, first quartile, median, third quartile, and maximum.

Box plots can be useful to display differences between populations without making any assumptions of the underlying statistical distribution: they are non-parametric. The spacings between the different parts of the box help indicate the degree of dispersion (spread) and skewness in the data, and identify outliers.

```
def boxplot(data: (String, Array[Double]), group: (String, Array[String]), properties: JsObject = JsObject()): JsObject
```

In [ ]:
show(boxplot("sepallength" -> iris("sepallength").toDoubleArray, "class" -> iris("class").toStringArray))

## Histogram

A histogram is a graphical representation of the distribution of numerical data. The range of values is divided into a series of consecutive, non-overlapping intervals/bins. The bins must be adjacent, and are usually equal size.
```
    def hist(x: (String, Array[Double]), k: Int, properties: JsObject = JsObject()): JsObject
``` 
where k is the number of bins (10 by default), or you can also specify an array of the breakpoints between bins.

Let's apply the histogram to an interesting data: the wisdom of crowds. The original experiment took place about a hundred years ago at a county fair in England. The fair had a guess the weight of the ox contest. Francis Galton calculated the average of all guesses, which is right to within one pound.

Recently, NPR Planet Money ran the experiment again. NPR posted a couple of pictures of a cow (named Penelope) and asked people to guess her weight. They got over 17,000 responses. The average of guesses was 1,287 pounds, which is pretty close to Penelope's weight 1,355 pounds.

In [ ]:
val cow = read.csv("data/npr/cow.txt", header = false)("V1").toDoubleArray
show(hist("Weight" -> cow.filter(_ <= 3500), 50))